### 문자열의 앞뒤에서 공백 문자(스페이스, 탭, 줄바꿈 등)를 제거

In [130]:
import pandas as pd

def strip_all_columns(df):
    """
    데이터프레임의 모든 열에 대해 strip 처리를 수행합니다.
    문자열의 앞뒤에서 공백 문자(스페이스, 탭, 줄바꿈 등)를 제거합니다.
    """
    for column in df.columns:
        if df[column].dtype == 'object':  # 문자열 데이터인 경우에만 적용
            df[column] = df[column].str.strip()
    return df

train_df = pd.read_csv('./data/train.csv')
train_df = strip_all_columns(train_df)

### Illegal characters 수정

In [131]:
import re

def fix_illegal_characters(text):
    """
    주어진 텍스트에서 특정 Illegal characters를 수정하고 연속된 공백을 하나로 바꿉니다.
    """
    if pd.isna(text):
        return text, []

    original_text = text
    fixed_chars = []

    # 1. '\u200b' (제로 너비 공백) 제거
    if '\u200b' in text:
        text = text.replace('\u200b', '')
        fixed_chars.append('\u200b')

    # 2. '\r' (캐리지 리턴) 을 '\n' (새 줄)으로 대체
    if '\r' in text:
        text = text.replace('\r', '\n')
        fixed_chars.append('\r')

    # 3. '\x08' (백스페이스) 제거
    if '\x08' in text:
        text = text.replace('\x08', '')
        fixed_chars.append('\x08')

    # 4. '\x1d' (그룹 분리자) 제거
    if '\x1d' in text:
        text = text.replace('\x1d', '')
        fixed_chars.append('\x1d')

    # 5. 연속된 새 줄 문자를 하나의 새 줄 문자로 대체
    text = re.sub(r'\n+', '\n', text)
    fixed_chars.append('\n+')

    # 6. 연속된 공백을 하나의 공백으로 대체
    if re.search(r' {2,}', text):
        text = re.sub(r' +', ' ', text)
        fixed_chars.append('multiple_spaces')

    # 수정된 문자가 있는 경우에만 변경된 것으로 간주
    if original_text != text:
        return text, list(set(fixed_chars))
    else:
        return text, []

def process_csv(df):
    """
    Illegal characters를 수정합니다.
    """
    if 'dialogue' not in df.columns:
        print("Error: 'dialogue' 열이 CSV 파일에 없습니다.")
        return

    total_fixes = 0
    fix_counts = {}

    def fix_and_count(text):
        nonlocal total_fixes
        fixed_text, fixed_chars = fix_illegal_characters(text)
        total_fixes += len(fixed_chars)
        for char in fixed_chars:
            fix_counts[char] = fix_counts.get(char, 0) + 1
        return fixed_text

    df['dialogue'] = df['dialogue'].apply(fix_and_count)
    df['summary'] = df['summary'].apply(fix_and_count)

    print(f"총 {total_fixes}개의 Illegal characters 또는 패턴이 수정되었습니다.")
    print("문자 또는 패턴별 수정 횟수:")
    for char, count in fix_counts.items():
        if char == 'multiple_spaces':
            print(f"  연속된 공백: {count}회")
        else:
            print(f"  {repr(char)}: {count}회")

    return df

train_df = process_csv(train_df)

총 2540개의 Illegal characters 또는 패턴이 수정되었습니다.
문자 또는 패턴별 수정 횟수:
  연속된 공백: 121회
  '\n+': 1270회
  '\u200b': 2회
  '\r': 1134회
  '\x08': 11회
  '\x1d': 2회


### 한글 모음/자음 처리

In [132]:
import pandas as pd
import re
from collections import Counter

def find_korean_jamo(text):
    """
    주어진 텍스트에서 한글 자음이나 모음을 찾아 반환합니다.
    
    :param text: 확인할 텍스트
    :return: 찾은 한글 자음/모음 리스트
    """
    # 한글 자음 모음 패턴
    jamo_pattern = re.compile('[ㄱ-ㅎㅏ-ㅣ]')
    return jamo_pattern.findall(text)

def check_korean_jamo_in_csv(df):
    """
    CSV 파일의 dialogue 열에서 한글 자음이나 모음이 포함된 대화를 찾습니다.
    
    :param file_path: CSV 파일 경로
    """
    if 'dialogue' not in df.columns:
        print("Error: 'dialogue' 열이 CSV 파일에 없습니다.")
        return
    
    # 한글 자음/모음이 포함된 대화 찾기
    all_jamo = []
    jamo_dialogues = []
    
    for _, row in df.iterrows():
        jamo = find_korean_jamo(row['dialogue'])
        if jamo:
            all_jamo.extend(jamo)
            jamo_dialogues.append((_, row['dialogue']))
    
    # 결과 출력
    print(f"총 대화 수: {len(df)}")
    print(f"한글 자음/모음이 포함된 대화 수: {len(jamo_dialogues)}")
    
    # 자음/모음 빈도 계산 및 출력
    jamo_freq = Counter(all_jamo)
    print("\n포함된 한글 자음/모음 목록 (빈도순):")
    for jamo, count in jamo_freq.most_common():
        print(f"{jamo}: {count}회")
    
    if jamo_dialogues:
        print("\n한글 자음/모음이 포함된 대화 예시 :")
        for idx, dialogue in jamo_dialogues:
            print(f"Index {idx}:")
            print(dialogue)
            print("포함된 자음/모음:", ", ".join(find_korean_jamo(dialogue)))
            print("-" * 50)

check_korean_jamo_in_csv(train_df)
check_korean_jamo_in_csv(pd.read_csv('./data/dev.csv'))
check_korean_jamo_in_csv(pd.read_csv('./data/test.csv'))

총 대화 수: 12457
한글 자음/모음이 포함된 대화 수: 7

포함된 한글 자음/모음 목록 (빈도순):
ㅋ: 4회
ㅇ: 1회
ㅣ: 1회
ㅏ: 1회
ㅍ: 1회
ㄷ: 1회

한글 자음/모음이 포함된 대화 예시 :
Index 3154:
#Person1#: 짐! 잘 지내? 
#Person2#: 찰리! 저게 너의 차야? 완전 못생겼네, 친구야! 
#Person1#: 멍청이가 되지마! 이건 1969년형 쉐비 임팔라야! 좀 고쳐야겠지만. 몇 달 안에 이 아기는 완전 멋질 거야! 
#Person2#: 아니야! 이걸 봐! 이게 바로 멋진 차야! 
#Person1#: 아쉽게도 운전자는 완전 바보야. 그런 차를 가질 수 있는 건 아빠가 그만큼 돈이 많아서야. 
#Person2#: 그가 이쪽으로 오고 있어, 차분하게 있어. 
#Person3#: 여러분! 내 차 어때요? 완전 멋지지 않아요? 
#Person1#: 맞아! 여자들이 너를 그 차로 돌아다니는 걸 보면 너에게 줄을 서게 될 거야. 
#Person3#: 정말 그럴까요? 
#Person2#: 확실해! 
#Person3#: 대박! 
#Person1#: 속았어! ㅋㅋ.. 완전 속았어. 
#Person3#: 넌 진짜 나쁜 놈이야, 찰리. 프롬 퀸과 재미있는 시간을 보낼 때, 마지막 웃음은 누가 낼지 봐. 
#Person2#: 친구야, 화내지 마!
포함된 자음/모음: ㅋ, ㅋ
--------------------------------------------------
Index 5385:
#Person1#: 제 영어 선생님이 저에게 오만과 편견을 읽으라고 하다니 믿을 수가 없어요!
#Person2#: 왜요! 그건 클래식이에요. 사실, 제가 가장 좋아하는 소설 중 하나예요.
#Person1#: 하지만 너무 오래됐어요.
#Person2#: 표지만 보고 책을 판단하지 마세요. 그 내용을 알고 있나요?
#Person1#: 아니요, 전혀 모르겠어요.
#Person2#: 먼저, 이것은 19세기 초 배경ㅇ로 설정된 로맨스 소설이에요.
#Pers

In [133]:
train_df.at[3154, 'dialogue'] = train_df.at[3154, 'dialogue'].replace('ㅋㅋ', '하하')
train_df.at[5385, 'dialogue'] = train_df.at[5385, 'dialogue'].replace('ㅇ', '으')
train_df.at[5429, 'dialogue'] = train_df.at[5429, 'dialogue'].replace('ㅋㅋ', '하하')
train_df.at[7201, 'dialogue'] = train_df.at[7201, 'dialogue'].replace('ㅏ', '가')
train_df.at[9677, 'dialogue'] = train_df.at[9677, 'dialogue'].replace('ㅍ', ' ')
train_df.at[12178, 'dialogue'] = train_df.at[12178, 'dialogue'].replace('ㄷ', '')

In [134]:
check_korean_jamo_in_csv(train_df)

총 대화 수: 12457
한글 자음/모음이 포함된 대화 수: 1

포함된 한글 자음/모음 목록 (빈도순):
ㅣ: 1회

한글 자음/모음이 포함된 대화 예시 :
Index 6942:
#Person1#: 여기가 제 사촌이 운영하는 코펠리니 가게예요. 
#Person2#: 'ㅣ'로 끝나는 이탈리아 이름 같네요. 
#Person1#: 솔직히 말해봐요. 당신 무솔리니를 생각하고 있죠. 
#Person2#: 아니요, 저는 예술 애호가라서 벨리니와 보티첼리를 더 생각하게 돼요! 
#Person1#: 아, 그렇죠. 이탈리아인들은 인체의 감각적인 형태를 사랑하죠. 
#Person2#: 우리 모두 그렇죠... 어! 당신 사촌이 선물용품을 팔아요? 
#Person1#: 네-모두 이탈리아에서 온 거예요. 성자들 아시죠? 그들은 로마 가톨릭에서 중요해요.
포함된 자음/모음: ㅣ
--------------------------------------------------


### Dialogue 패턴 오류 수정

##### Dialogue 패턴 분석

In [135]:
from collections import Counter

def check_dialogue_pattern(dialogue):
    """
    대화 패턴을 확인하고 예외를 찾습니다.
    발화자는 #PersonN# 형식을 따릅니다 (N은 숫자).
    """
    if pd.isna(dialogue):
        return False, "대화가 비어 있습니다."
    
    lines = dialogue.split('\n')
    speakers = []
    for i, line in enumerate(lines, 1):
        # 빈 줄 무시
        if not line.strip():
            continue
        
        # 패턴 확인
        match = re.match(r'^(#Person\d+#): (.+)', line)
        if not match:
            return False, f"{i}번째 줄이 올바른 형식이 아닙니다: {line}"
        
        speaker, content = match.groups()
        speakers.append(speaker)
        
        # 발화 내용이 비어있는지 확인
        if not content.strip():
            return False, f"{i}번째 줄의 대화 내용이 비어 있습니다: {line}"
    
    # 발화자 순서 확인
    if len(speakers) < 2:
        return False, "대화에 최소 두 명의 발화자가 필요합니다."
    
    # for i in range(1, len(speakers)):
    #     if speakers[i] == speakers[i-1]:
    #         return False, f"{i+1}번째 발화자가 이전 발화자와 같습니다: {speakers[i]}"
    
    # # 발화자 번호의 연속성 확인
    # speaker_numbers = sorted(set(int(re.search(r'\d+', s).group()) for s in speakers))
    # if speaker_numbers != list(range(min(speaker_numbers), max(speaker_numbers) + 1)):
    #     return False, f"발화자 번호가 연속적이지 않습니다: {speakers}"
    
    return True, "올바른 대화 패턴입니다."

def analyze_dialogue_patterns(df, column='dialogue'):
    """
    데이터프레임의 대화 열을 분석하고 패턴 예외를 찾습니다.
    결과를 CSV 파일로 저장합니다.
    """
    df = df.copy()
    
    results = df[column].apply(check_dialogue_pattern)
    df['is_valid_pattern'] = results.apply(lambda x: x[0])
    df['pattern_error'] = results.apply(lambda x: x[1])
    
    exceptions = df[~df['is_valid_pattern']]
    
    print(f"\n대화 패턴 분석 결과 ({column} 열):")
    print(f"전체 대화 수: {len(df)}")
    print(f"패턴 예외 수: {len(exceptions)}")
    
    if not exceptions.empty:
        print("\n패턴 예외 예시 (최대 5개):")
        for idx, row in exceptions.head().iterrows():
            print(f"\nIndex {idx}:")
            print(f"대화:\n{row[column]}")
            print(f"오류: {row['pattern_error']}")
    
    # 발화자 통계
    all_speakers = [speaker for dialogue in df[column].dropna() for speaker in re.findall(r'#Person\d+#', dialogue)]
    speaker_counts = Counter(all_speakers)
    
    print("\n발화자 통계:")
    print(f"유니크한 발화자 수: {len(speaker_counts)}")
    print("가장 흔한 발화자 Top :")
    for speaker, count in speaker_counts.most_common():
        print(f"{speaker}: {count}회")
    
    return exceptions

def analyze_dialogue_pattern(df):
    # 대화 패턴 분석
    dialogue_exceptions = analyze_dialogue_patterns(df)
    dialogue_exceptions
    return dialogue_exceptions

print("Train 대화 패턴 분석:")
dialogue_exceptions = analyze_dialogue_pattern(train_df)
dialogue_exceptions.to_csv(f'./data/dialogue_exceptions.csv', index=False)
print("\nDev 대화 패턴 분석:")
analyze_dialogue_pattern(pd.read_csv('./data/dev.csv'))
print("\nTest 대화 패턴 분석:")
analyze_dialogue_pattern(pd.read_csv('./data/test.csv'))

Train 대화 패턴 분석:

대화 패턴 분석 결과 (dialogue 열):
전체 대화 수: 12457
패턴 예외 수: 55

패턴 예외 예시 (최대 5개):

Index 767:
대화:
#Person1#: 저의 명함이 곧 다 떨어질 것 같아요. 새로운 것이 필요해요. 
#Person2#: 원하는 만큼 인쇄할 수 있어요. 몇 장이 필요한지만 말씀해주세요. 
#Person1#: 이천 장이면 올해를 보낼 수 있을 것 같아요. 
#Person2#: 시작하실 수 있도록 양식을 드릴게요. 
#Person1#: 제 이전 명함이 완벽해서, 그냥 그대로 복사해주기만 하면 돼요. 
#Person2#: 우리가 당신의 이전 명함을 얼마나 잘 복제하는지에 대해 매우 만족하실 거예요. 
#Person1#: . . . 여기 있어요 
#Person2#: 감사합니다. 다음 주 수요일에 주문하신 걸 찾으러 오세요. 
#Person1#: 죄송하지만, 3일 안에 만들어 줄 수 있을까요? 
#Person2#: 조금 더 비용을 지불하는 것이 괜찮다면, 전혀 문제 없습니다. 
#Person3#:
오류: 11번째 줄이 올바른 형식이 아닙니다: #Person3#:

Index 839:
대화:
#Person1#: 안녕하세요, 저희 은행에 오신 것을 환영합니다. 오늘은 무엇을 도와드릴까요?
#Person2#: 정기예금에 대해 조언이 필요합니다.
#Person1#: 무슨 문제가 있으신가요?
#Person2#: 아니요, 문제 없습니다. 그냥 정기예금 중 하나가 만기가 되어서 어떻게 처리해야 할지 잘 모르겠어요.
#Person1#: 그렇군요. 두 가지 선택이 있는데, 갱신하거나 교환할 수 있습니다. 어느 쪽을 선호하시나요?
#Person2#: 음.. 지금까지 서비스에 만족하고 있으니 갱신하고 싶어요.
#Person1#: 문제없습니다. 하지만 만기 시 계정 갱신이라는 새로운 서비스가 도입되었음을 알려드려도 될까요?
#Person2#: 네. . .
#사람1만기 시 계정 갱신 서비스는 만기일이 다가오면 자동으로 갱신

,fname,dialogue,is_valid_pattern,pattern_error
434,test_434,"#Person1#: 헤이, 앤드류! 앤...? 앤드류.\n#Person2#: 뭐?\...",False,3번째 줄이 올바른 형식이 아닙니다: #Person1#:앤드류.


##### 내용이 없는 줄을 제거

In [136]:
def remove_empty_dialogue_lines(dialogue):
    """
    대화에서 발화자만 있고 내용이 없는 줄을 제거합니다.

    :param dialogue: 원본 대화 문자열
    :return: 수정된 대화 문자열, 제거된 줄 수
    """
    lines = dialogue.strip().split('\n')
    filtered_lines = []
    removed_lines = 0

    for line in lines:
        # 발화자 태그와 내용 분리
        match = re.match(r'^(#[^#]+#:)\s*(.*)$', line.strip())
        if match:
            speaker, content = match.groups()
            if content.strip():  # 내용이 있는 경우만 포함
                filtered_lines.append(line.strip())
            else:
                removed_lines += 1
        else:
            filtered_lines.append(line.strip())  # 발화자 태그가 없는 줄은 그대로 유지

    return '\n'.join(filtered_lines), removed_lines

def delete_empty_line(df):
    """
    CSV 파일의 dialogue 열에서 빈 대화 줄을 제거합니다.
    """

    if 'dialogue' not in df.columns:
        print("Error: 'dialogue' 열이 CSV 파일에 없습니다.")
        return

    total_removed_lines = 0
    modified_dialogues = []

    def apply_removal(dialogue):
        nonlocal total_removed_lines
        cleaned_dialogue, removed = remove_empty_dialogue_lines(dialogue)
        total_removed_lines += removed
        if removed > 0:
            modified_dialogues.append((dialogue, cleaned_dialogue))
        return cleaned_dialogue

    df['dialogue'] = df['dialogue'].apply(apply_removal)

    print(f"\n총 {len(modified_dialogues)}개의 대화가 수정되었습니다.")

    for original, modified in modified_dialogues:
        print("\n원본 :")
        print(original)
        print("수정본 :")
        print(modified)
    
    return df

train_df = delete_empty_line(train_df)


총 2개의 대화가 수정되었습니다.

원본 :
#Person1#: 저의 명함이 곧 다 떨어질 것 같아요. 새로운 것이 필요해요. 
#Person2#: 원하는 만큼 인쇄할 수 있어요. 몇 장이 필요한지만 말씀해주세요. 
#Person1#: 이천 장이면 올해를 보낼 수 있을 것 같아요. 
#Person2#: 시작하실 수 있도록 양식을 드릴게요. 
#Person1#: 제 이전 명함이 완벽해서, 그냥 그대로 복사해주기만 하면 돼요. 
#Person2#: 우리가 당신의 이전 명함을 얼마나 잘 복제하는지에 대해 매우 만족하실 거예요. 
#Person1#: . . . 여기 있어요 
#Person2#: 감사합니다. 다음 주 수요일에 주문하신 걸 찾으러 오세요. 
#Person1#: 죄송하지만, 3일 안에 만들어 줄 수 있을까요? 
#Person2#: 조금 더 비용을 지불하는 것이 괜찮다면, 전혀 문제 없습니다. 
#Person3#:
수정본 :
#Person1#: 저의 명함이 곧 다 떨어질 것 같아요. 새로운 것이 필요해요.
#Person2#: 원하는 만큼 인쇄할 수 있어요. 몇 장이 필요한지만 말씀해주세요.
#Person1#: 이천 장이면 올해를 보낼 수 있을 것 같아요.
#Person2#: 시작하실 수 있도록 양식을 드릴게요.
#Person1#: 제 이전 명함이 완벽해서, 그냥 그대로 복사해주기만 하면 돼요.
#Person2#: 우리가 당신의 이전 명함을 얼마나 잘 복제하는지에 대해 매우 만족하실 거예요.
#Person1#: . . . 여기 있어요
#Person2#: 감사합니다. 다음 주 수요일에 주문하신 걸 찾으러 오세요.
#Person1#: 죄송하지만, 3일 안에 만들어 줄 수 있을까요?
#Person2#: 조금 더 비용을 지불하는 것이 괜찮다면, 전혀 문제 없습니다.

원본 :
#Person1#: 안녕하세요, 저는 조지입니다. 오늘 저녁 담당 웨이터가 될게요. 주문하실 준비가 되셨나요, 아니면 조금 더 시간이 필요하신가요? 
#Person2#: 지금 주문할게요. 로스

##### 발화자 패턴 수정

In [137]:
import random

def fix_dialogue_speakers(dialogue):
    """
    대화에서 발화자 태그를 수정합니다.
    1. ':' 를 기준으로 발화자와 내용을 구분합니다.
    2. 발화자 패턴이 '^(#Person\d+#): (.+)' 이 아닐 경우, 첫 번째 토큰을 다음 발화자 태그로 교체합니다.

    :param dialogue: 원본 대화 문자열
    :return: 수정된 대화 문자열, 수정 정보 딕셔너리
    """
    lines = dialogue.strip().split('\n')
    fixed_lines = []
    pattern = re.compile(r'^(#Person\d+#): (.+)')
    current_speaker = "#Person1#"
    modified = False

    for line in lines:
        match = pattern.match(line)
        if match:
            # 이미 올바른 패턴인 경우
            current_speaker = match.group(1)  # 패턴에서 발화자 추출
            fixed_lines.append(line)
        else:
            # 올바른 패턴이 아닌 경우
            parts = line.split(':', 1)
            if len(parts) == 2:
                _, content = parts
                content = content.strip()
            else:
                content = line.strip()
            
            fixed_line = f"{current_speaker}: {content}"
            fixed_lines.append(fixed_line)
            modified = True

        # 다음 발화자 준비
        current_speaker = f"#Person{2 if current_speaker == '#Person1#' else 1}#"

    fixed_dialogue = '\n'.join(fixed_lines)

    return fixed_dialogue, {"modified": modified}

def modify_dialogue_speakers(df):
    """
    CSV 파일의 dialogue 열에 fix_dialogue_speakers 함수를 적용하고,
    수정된 대화와 수정되지 않은 대화를 분리합니다.
    """

    if 'dialogue' not in df.columns:
        print("Error: 'dialogue' 열이 CSV 파일에 없습니다.")
        return

    modified_count = 0
    unmodified_count = 0
    modified_examples = []
    unmodified_examples = []

    def apply_fix(row):
        nonlocal modified_count, unmodified_count
        original_dialogue = row['dialogue']
        fixed_dialogue, info = fix_dialogue_speakers(original_dialogue)
        if info['modified']:
            modified_count += 1
            modified_examples.append((original_dialogue, fixed_dialogue))
            return fixed_dialogue
        else:
            unmodified_count += 1
            unmodified_examples.append(original_dialogue)
            return original_dialogue

    df['dialogue'] = df.apply(lambda row: apply_fix(row), axis=1)

    print(f"처리 완료:")
    print(f"수정된 대화 수: {modified_count}")
    print(f"수정되지 않은 대화 수: {unmodified_count}")

    # 수정된 대화 예시 출력
    print("\n수정된 대화 예시 (2개):")
    for original, fixed in random.sample(modified_examples, min(2, len(modified_examples))):
        print("원본:")
        print(original)
        print("\n수정됨:")
        print(fixed)
        print("-" * 50)

    # 수정되지 않은 대화 예시 출력
    print("\n수정되지 않은 대화 예시 (2개):")
    for original in random.sample(unmodified_examples, min(2, len(unmodified_examples))):
        print(original)
        print("-" * 50)

    return df

train_df = modify_dialogue_speakers(train_df)

처리 완료:
수정된 대화 수: 53
수정되지 않은 대화 수: 12404

수정된 대화 예시 (2개):
원본:
#Person1#: 방금 집어든 그 책은 뭔가요?
#Person2#: 스미스 교수님이 수업에서 사용하는 사회학 교재에요.
#Person1#: 그 과목을 통과하려면 그 책을 읽어야 할 거예요.
#Person2#: 하지만 너무 비싸요. 제가 감당할 수 없어요.
#Person1#: 얼마나 하나요?
#Person2#: 40달러입니다.
#Person1#: 여기 중고책 코너를 확인해 보셨나요? 아마 있을지도 몰라요.
#Person2#: 없어요, 물어봤어요.
#Person1#: 도서관에서 빌리면 어떨까요?
#Person2#: 농담하시는 거죠? 몇 달 동안 시도해봤지만 항상 대출 중이에요. 이 과목에는 수강생이 45명이 넘는데 다들 책을 원해요.
#Person1#: 들어봐요, 제 룸메이트인 헨리 알지요? 작년에도 같은 수업을 들었고, 저는 헨리가 그 책을 가지고 있다고 생각해요. 제가 그에게 빌려줄 수 있는지 물어볼게요.
#Person2#:오, 톰, 그럼 모든 게 해결되겠네요. 정말 친절하시네요.
#Person1#: 천만에요.

수정됨:
#Person1#: 방금 집어든 그 책은 뭔가요?
#Person2#: 스미스 교수님이 수업에서 사용하는 사회학 교재에요.
#Person1#: 그 과목을 통과하려면 그 책을 읽어야 할 거예요.
#Person2#: 하지만 너무 비싸요. 제가 감당할 수 없어요.
#Person1#: 얼마나 하나요?
#Person2#: 40달러입니다.
#Person1#: 여기 중고책 코너를 확인해 보셨나요? 아마 있을지도 몰라요.
#Person2#: 없어요, 물어봤어요.
#Person1#: 도서관에서 빌리면 어떨까요?
#Person2#: 농담하시는 거죠? 몇 달 동안 시도해봤지만 항상 대출 중이에요. 이 과목에는 수강생이 45명이 넘는데 다들 책을 원해요.
#Person1#: 들어봐요, 제 룸메이트인 헨리 알지요? 작년에도 같은 수업을 들었고, 저는 헨리가 

### 중복 된 dialogue, summary 확인

##### dialogue, summary 모두 중복인 행 제거

In [138]:
def clear_duplicated(df):
    # 중복 확인
    print("중복 제거 전 데이터 개수:", len(df))
    duplicates = df[df.duplicated(subset=['dialogue', 'summary'], keep=False)]
    print("중복된 dialogue, summary 개수:", len(duplicates))

    # 중복 제거
    df.drop_duplicates(subset=['dialogue', 'summary'], inplace=True)
    print("중복 제거 후 데이터 개수:", len(df))

clear_duplicated(train_df)

중복 제거 전 데이터 개수: 12457
중복된 dialogue, summary 개수: 4
중복 제거 후 데이터 개수: 12455


##### 중복 summary 확인

In [139]:
# summary만 중복 확인
duplicates = train_df[train_df.duplicated(['summary'], keep=False)]
print("중복된 summary 개수:", len(duplicates))

duplicates = duplicates.sort_values(by='summary', ascending=True)
print(duplicates)

중복된 summary 개수: 27
             fname                                           dialogue  \
8785    train_8785  #Person1#: 음악 좋아해요?\n#Person2#: 그건 상황에 따라 다르죠....   
8754    train_8754  #Person1#: 음악 좋아하시나요?\n#Person2#: 그건 상황에 따라 다르...   
875      train_875  #Person1#: 안녕하세요, 파커. 어떻게 지내세요?\n#Person2#: 불만...   
3654    train_3654  #Person1#: 안녕, 파커. 별일 없지?\n#Person2#: 그럭저럭 잘 지...   
7550    train_7550  #Person1#: 왜 숙제를 안 하고 있니?\n#Person2#: 나중에 할게요,...   
7551    train_7551  #Person1#: 좋아, 최신 지도를 가지고 있어. 여기서 공항까지의 경로를 확인...   
1076    train_1076  #Person1#: 실례지만 잠시만 시간을 내 주실 수 있으세요?\n#Person2...   
5673    train_5673  #Person1#: 실례합니다, 선생님. 잠시 시간을 내주실 수 있으신가요?\n#P...   
8975    train_8975  #Person1#: 저는 기독교인이 아닙니다. 그런데 왜 미국인들이 그런 것을 믿는...   
11608  train_11611  #Person1#: 저는 기독교인이 아닙니다. 그런데 왜 미국인들이 그런 것을 믿는...   
6189    train_6189  #Person1#: 버스 패스는 얼마인가요?\n#Person2#: 월간 패스는 65...   
6640    train_6640  #Person1#: 버스 패스는 얼마인가요?\n#Person2#: 월간 패스의 경우...   
830      train_830  #Person1#: 내

##### 잘못 요약된 summary 삭제

In [140]:
values_to_delete = ['train_7551', 'train_830', 'train_7528', 'train_7529', 'train_7557'] 
train_df = train_df[~train_df['fname'].isin(values_to_delete)]

train_df

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스
...,...,...,...,...
12452,train_12455,#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2...,탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12456,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12458,#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직


##### 중복 dialogue 확인

In [141]:
# dialogue만 중복 확인
duplicates = train_df[train_df.duplicated(['dialogue'], keep=False)]
print("중복된 dialogue 개수:", len(duplicates))

duplicates = duplicates.sort_values(by='dialogue', ascending=True)
print(duplicates)

중복된 dialogue 개수: 104
             fname                                           dialogue  \
12418  train_12421  #Person1#: 그들은 우리에게 바다 전망을 약속했어요.\n#Person2#: ...   
11536  train_11539  #Person1#: 그들은 우리에게 바다 전망을 약속했어요.\n#Person2#: ...   
5942    train_5942  #Person1#: 그래서, 수잔, 이번 토요일에 계획이 있나요?\n#Person2...   
6409    train_6409  #Person1#: 그래서, 수잔, 이번 토요일에 계획이 있나요?\n#Person2...   
2131    train_2131  #Person1#: 근육을 스트레칭하고 건강해지는 데 도움이 되는 몇 가지 운동을 ...   
...            ...                                                ...   
7294    train_7294  #Person1#: 헤이, 빌. 새 프로젝트로 널 독일로 보낼 거라는 소식을 들었어...   
4946    train_4946  #Person1#: 현지 유명 영매에 대한 이 신문 기사를 보세요. 기사에 따르면 ...   
4834    train_4834  #Person1#: 현지 유명 영매에 대한 이 신문 기사를 보세요. 기사에 따르면 ...   
8092    train_8092  #Person1#: 환영합니다!\n#Person2#: 맥주 한 병 주시겠어요?\n#...   
5495    train_5495  #Person1#: 환영합니다!\n#Person2#: 맥주 한 병 주시겠어요?\n#...   

                                                 summary        topic  
12418  #Person2#는 #Person1#이 끔

##### 중복 dialogue의 summary 와의 유사도 확인

In [142]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from konlpy.tag import Okt
from tqdm import tqdm

def preprocess_korean_text(text):
    okt = Okt()
    tokens = okt.morphs(text)
    return ' '.join(tokens)

def keyword_matching(dialogue, summary):
    vectorizer = CountVectorizer()
    dialogue = preprocess_korean_text(dialogue)
    summary = preprocess_korean_text(summary)
    corpus = [dialogue, summary]
    vectors = vectorizer.fit_transform(corpus)
    keywords = vectorizer.get_feature_names_out()
    dialogue_vector = vectors[0].toarray()[0]
    summary_vector = vectors[1].toarray()[0]
    common_keywords = [word for word, count1, count2 in zip(keywords, dialogue_vector, summary_vector) if count1 > 0 and count2 > 0]
    return len(common_keywords) / len(keywords)

def sentence_similarity(dialogue, summary):
    model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
    dialogue_embedding = model.encode(dialogue)
    summary_embedding = model.encode(summary)
    similarity = cosine_similarity([dialogue_embedding], [summary_embedding])[0][0]
    return similarity

def calc(df):
    # 키워드 매칭과 문장 유사도 계산
    tqdm.pandas(desc="Keyword Matching")
    df['keyword_match_ratio'] = df.progress_apply(lambda x: keyword_matching(x['dialogue'], x['summary']), axis=1)
    # tqdm.pandas(desc="Sentence Similarity")  
    # df['sentence_similarity'] = df.progress_apply(lambda x: sentence_similarity(x['dialogue'], x['summary']), axis=1)
    
    return df
   
duplicates = calc(duplicates)

Keyword Matching: 100%|██████████| 104/104 [00:03<00:00, 28.98it/s]


In [143]:
duplicates = duplicates.sort_values(by=['keyword_match_ratio'], axis=0)
duplicates

duplicates.to_csv('./data/train_duped_dialogue.csv', index=False)

##### 잘못 요약된 dialogue 삭제

In [144]:
values_to_delete = ['train_10066', 'train_10062'] 
train_df = train_df[~train_df['fname'].isin(values_to_delete)]
train_df

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스
...,...,...,...,...
12452,train_12455,#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2...,탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12456,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12458,#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직


##### 중복 외에도 잘못 요약된 것이 없는지 확인

In [145]:
import pandas as pd
import re
from konlpy.tag import Okt
from collections import Counter

okt = Okt()

def extract_person_names(text):
    """
    텍스트에서 '은', '는', '이', '가' 앞에 오는 단어 중 명사, 대명사, 고유명사를 추출합니다.
    
    :param text: 분석할 텍스트
    :return: 추출된 이름 리스트
    """
    # OKR의 형태소 분석 결과 얻기
    pos_tags = okt.pos(text)
    
    names = []
    specific_particles = ['은', '는', '이', '가']
    
    for i in range(len(pos_tags) - 1):
        word, tag = pos_tags[i]
        # print(i)
        # print(tag)
        # print(word)
        next_word, next_tag = pos_tags[i + 1]
        
        # 현재 단어가 명사이고 다음 단어가 특정 조사인 경우
        if tag in ['Noun'] and next_word in specific_particles:
            names.append(word)
    
    return names

def find_mismatched_names(df):
    """
    CSV 파일에서 summary의 이름이 dialogue에 없는 행을 찾습니다.
    
    :param input_file: 입력 CSV 파일 경로
    :return: 불일치하는 행들의 DataFrame, 불일치하는 이름들의 Counter
    """

    if 'dialogue' not in df.columns or 'summary' not in df.columns:
        print("Error: 'dialogue' 또는 'summary' 열이 CSV 파일에 없습니다.")
        return pd.DataFrame(), Counter()
    
    mismatched_rows = []
    all_mismatched_names = []
    
    for index, row in df.iterrows():
        summary_names = extract_person_names(row['summary'])
        
        mismatched_names = [name for name in summary_names if name not in row['dialogue']]
        
        if mismatched_names:
            mismatched_rows.append({
                'index': index,
                'summary': row['summary'],
                'dialogue': row['dialogue'],
                'mismatched_names': ', '.join(mismatched_names)
            })
            all_mismatched_names.extend(mismatched_names)
    
    result_df = pd.DataFrame(mismatched_rows)
    name_counts = Counter(all_mismatched_names)
    
    return result_df, name_counts

mismatched_df, name_counts = find_mismatched_names(train_df)

if not mismatched_df.empty:
    mismatched_df.to_csv('./data/train_mismatched.csv', index=False)
    print(f"불일치하는 {len(mismatched_df)}개의 행을 찾았습니다.")
    
    print("\n불일치하는 행의 예시 (최대 5개):")
    for _, row in mismatched_df.head().iterrows():
        print(f"Index: {row['index']}")
        print(f"Summary: {row['summary']}")
        print(f"Dialogue: {row['dialogue']}")
        print(f"불일치하는 이름: {row['mismatched_names']}")
        print("-" * 50)
    
    print("\n불일치하는 이름들의 등장 횟수 (적은 순):")
    for name, count in sorted(name_counts.items(), key=lambda x: x[1]):
        print(f"{name}: {count}회")
else:
    print("불일치하는 행을 찾지 못했습니다.")

불일치하는 1555개의 행을 찾았습니다.

불일치하는 행의 예시 (최대 5개):
Index: 7
Summary: 주디 리아오는 회계 보조 자리에 지원하고 있습니다. 그녀는 #Person1#에게 이력서를 받았는지 물었고, #Person1#은 그녀에게 확인해주었습니다. #Person1#은 주디에게 다른 것을 보낼 필요가 없다고 말하고, 일주일이나 두 주 후에 지원자들에게 전화를 걸기 시작할 것이라고 말했습니다.
Dialogue: #Person1#: 도와드릴 일이 있나요?
#Person2#: 네. 저는 지난 주말에 이력서를 보냈습니다. 회계 보조 자리에 지원하려고 합니다.
#Person1#: 이름을 알려주실 수 있나요?
#Person2#: 제 이름은 주디 리아오입니다. 엘 아이 에이 오로 씁니다.
#Person1#: 알겠습니다. . . 지원서에 대해 특별히 궁금한 점이 있나요?
#Person2#: 아니요. 근처에 있어서 제 이력서를 받았는지 확인하러 들렸습니다.
#Person1#: 아, 문제 없습니다. 잠시만 기다려주시면 확인해보겠습니다. 주디 리아오. 찾아보겠습니다. . . 네, 여기 있네요.주디 리아오. 이력서를 받았습니다.
#Person2#: 감사합니다.
#Person1#: 다른 도움이 필요한 것이 있나요?
#Person2#: 네, 아마도요. 신문 광고에는 이력서, 자기소개서, 그리고 추천서 두 통을 원한다고 써 있었습니다. 그것들을 모두 봉투에 넣어 보냈습니다. 다른 것을 보내야 할까요?
#Person1#: 아니요, 그것들만 있으면 충분합니다. 그것들이 포함되어 있다면 충분합니다.
#Person2#: 면접 일정이 언제 시작되는지 아시나요?
#Person1#: 그건 정확히 모르겠습니다. 하지만 아직도 이력서를 받고 있습니다. 아마 일주일이나 두 주 후에 지원자들에게 전화를 걸기 시작할 것 같습니다.
#Person2#: 알겠습니다. 도와주셔서 정말 감사합니다. 많은 도움이 되었습니다.
#Person1#: 추가적인 질문이 있으시면 언제든지 전화하실 수 있습니다.


In [146]:
# 사람 이름으로 수기 검토해본 결과 영어 이름이 대화, 요약에 각각 한글로 번역되면서 달라진 것으로 보임.
# ex) steve -> 대화:스티븐, 요약:스티브

### 영어로 작성된 행 확인

In [147]:
korean = '[가-힣]'

# 한글이 포함되지 않은 행을 필터링
# 한글 범위: 가-힣
english_only_df = train_df[~train_df['dialogue'].str.contains(korean, na=False)]
print(f'* dialogue\n{english_only_df}')

english_only_df = train_df[~train_df['summary'].str.contains(korean, na=False)]
print(f'\n* summary\n')
english_only_df

* dialogue
Empty DataFrame
Columns: [fname, dialogue, summary, topic]
Index: []

* summary



,fname,dialogue,summary,topic
6812,train_6812,"#Person1#: 안녕하세요, 할머니. 다시 만나서 정말 좋아요.\n#Person...",Grandma gives Jack a racing car as a gift. The...,할머니를 데리러 가다
10896,train_10896,#Person1#: 기분이 어때?\n#Person2#: 나빠. 코가 계속 흐르고 이...,Daniel got sick and Tom envies Daniel because ...,비밀을 유지하기
10897,train_10897,#Person1#: 유럽 투어에 대한 정보를 좀 주실 수 있나요?\n#Person2...,#Person2#gives #Person1# some information on t...,유럽 투어
10900,train_10900,"#Person1#: 좋아요, 테일러 씨, 시작해봅시다. 먼저, 마지막 직장에 대해 ...",#Person2# is interviewing Mr. Taylor. #Person2...,직장 면접


##### 영어 summary 삭제

In [148]:
train_df = train_df[train_df['summary'].str.contains(korean, na=False)]

# 전처리 된 train 데이터셋 저장

In [149]:
train_df.reset_index(drop=True, inplace=True)
train_df

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스
...,...,...,...,...
12439,train_12455,#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2...,탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12440,train_12456,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12441,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12442,train_12458,#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직


In [ ]:
train_df.to_csv('./data/train_cleaned.csv', index=False)